# Demonstration for mlflow REST API

This notebook demonstrates use of the mlflow tracking REST api to retrieve results from mlflow experiments and place them into a pandas dataframe.

In [1]:
from __future__ import print_function

In [2]:
import pandas as pd
import numpy as np
import os
import os.path
import requests
import socket

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.tracking
import mlflow.sklearn

## set up to invoke mlflow tracking REST api

In [3]:
# Assumes MFLOW_TRACKING_URI is set

REST_API_URL = os.environ['MLFLOW_TRACKING_URI'] + '/api/2.0/preview/mlflow'

print(REST_API_URL)

http://mlflow_tracker:5000/api/2.0/preview/mlflow


## List all experiments

In [4]:
r = requests.get(REST_API_URL + '/experiments/list')
experiment_list = r.json()
experiment_list

{'experiments': [{'artifact_location': '/artifacts/0',
   'lifecycle_stage': 'active',
   'name': 'mlflow_demo1',
   'experiment_id': '0'},
  {'artifact_location': '/artifacts/1',
   'lifecycle_stage': 'active',
   'name': 'mlflow_demo2',
   'experiment_id': '1'}]}

## Retrieve data for specified experiment

In [5]:
r = requests.get(REST_API_URL + '/experiments/get',
                 json={'experiment_id':'1'})
experiment_data = r.json()


In [6]:
experiment_data['experiment']

{'artifact_location': '/artifacts/1',
 'lifecycle_stage': 'active',
 'name': 'mlflow_demo2',
 'experiment_id': '1'}

In [7]:
experiment_metadata = {'experiment_name':experiment_data['experiment']['name'],
                       'experiment_id' : experiment_data['experiment']['experiment_id']}

## Extract experiment results to pandas dataframe

### Retrieve desired experiment attributes

In [8]:
df1 = pd.DataFrame([{k:experiment_data['experiment'][k] for k in ['name','experiment_id']}])
df1

,experiment_id,name
0,1,mlflow_demo2


### Extract run data (metrics, params and tags) 

In [9]:
def extract_run_data(r):
    # seed dataframe row with experiment attributes
    ans = experiment_metadata.copy()
    
    # add run uuid
    ans.update({'run_uuid': r['run_uuid']})
    
    # retrieve metrics, params and tags from the run
    r1 = requests.get(REST_API_URL + '/runs/get',
                      json={'run_uuid': r['run_uuid']})
    run_data = r1.json()['run']['data']
    
    # populate dataframe row with metrics, params and tags 
    for k in run_data.keys():    
        try:
            data = {k+'_'+x['key']:x['value'] for x in run_data[k]}
        except:
            data = {}

        ans.update(data)
        
    # return the dataframe row
    return ans

In [10]:
df = pd.DataFrame([extract_run_data(r) for r in experiment_data['runs']])
df

,experiment_id,experiment_name,metrics_r2,params_algorithm,params_alpha,params_data_set_type,params_hidden_layer_sizes,params_l1_ratio,params_learning_rate,params_max_depth,params_max_iter,params_n_estimators,params_random_state,run_uuid,tags_data_set_type,tags_estimator,tags_mlflow.source.name,tags_mlflow.source.type
0,1,mlflow_demo2,0.377038,RandomForestRegressor,NaN,raw values,NaN,NaN,NaN,7,NaN,279,13,04435977636f4e86b7c9befcb6e562f5,raw values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
1,1,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,raw values,NaN,NaN,NaN,7,NaN,279,13,2b9949cae44e45b7961459deaa89eaa4,raw values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
2,1,mlflow_demo2,0.377560,RandomForestRegressor,NaN,standardized values,NaN,NaN,NaN,7,NaN,279,13,aa9478ceea7e4cfbad36b01f14fcd769,standardized values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
3,1,mlflow_demo2,0.377378,RandomForestRegressor,NaN,Min/Max values,NaN,NaN,NaN,7,NaN,279,13,e15aa009827041de922aae35b88a6376,Min/Max values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
4,1,mlflow_demo2,0.500909,XGBRegressor,NaN,standardized values,NaN,NaN,0.058512491882074746,6,NaN,453,13,f77893521db04587a4695e8cd83a2bf2,standardized values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
5,1,mlflow_demo2,0.500365,XGBRegressor,NaN,raw values,NaN,NaN,0.058512491882074746,6,NaN,453,13,61402da9a2194d3cb9ba3cdcb4c18645,raw values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
6,1,mlflow_demo2,0.304265,MLPRegressor,NaN,Min/Max values,"(20,)",NaN,NaN,NaN,676,NaN,13,38d566963bf34d32b97e2d510a4b0908,Min/Max values,from sklearn.neural_network import MLPRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
7,1,mlflow_demo2,0.499639,XGBRegressor,NaN,Min/Max values,NaN,NaN,0.058512491882074746,6,NaN,453,13,5e5cbaea1bba4726989b798c1b655731,Min/Max values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
8,1,mlflow_demo2,0.220901,ElasticNet,0.009412770080969413,Min/Max values,NaN,0.35833378270496974,NaN,NaN,NaN,NaN,13,0f3698a6818c4c5683cd500a28337749,Min/Max values,from sklearn.linear_model import ElasticNet,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
9,1,mlflow_demo2,0.315244,MLPRegressor,NaN,standardized values,"(10, 10)",NaN,NaN,NaN,1343,NaN,13,b7863ba8b7c44c5b90d65432bd15a56b,standardized values,from sklearn.neural_network import MLPRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL


In [11]:
df[['params_alpha','params_l1_ratio','params_learning_rate','params_max_depth']] = \
    df[['params_alpha','params_l1_ratio','params_learning_rate','params_max_depth']].apply(pd.to_numeric)

In [12]:
df.dtypes

experiment_id                 object
experiment_name               object
metrics_r2                   float64
params_algorithm              object
params_alpha                 float64
params_data_set_type          object
params_hidden_layer_sizes     object
params_l1_ratio              float64
params_learning_rate         float64
params_max_depth             float64
params_max_iter               object
params_n_estimators           object
params_random_state           object
run_uuid                      object
tags_data_set_type            object
tags_estimator                object
tags_mlflow.source.name       object
tags_mlflow.source.type       object
dtype: object

## Display experiment results sorted by metric

In [13]:
df.sort_values('metrics_r2',ascending=False)

,experiment_id,experiment_name,metrics_r2,params_algorithm,params_alpha,params_data_set_type,params_hidden_layer_sizes,params_l1_ratio,params_learning_rate,params_max_depth,params_max_iter,params_n_estimators,params_random_state,run_uuid,tags_data_set_type,tags_estimator,tags_mlflow.source.name,tags_mlflow.source.type
4,1,mlflow_demo2,0.500909,XGBRegressor,NaN,standardized values,NaN,NaN,0.058512,6.0,NaN,453,13,f77893521db04587a4695e8cd83a2bf2,standardized values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
5,1,mlflow_demo2,0.500365,XGBRegressor,NaN,raw values,NaN,NaN,0.058512,6.0,NaN,453,13,61402da9a2194d3cb9ba3cdcb4c18645,raw values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
7,1,mlflow_demo2,0.499639,XGBRegressor,NaN,Min/Max values,NaN,NaN,0.058512,6.0,NaN,453,13,5e5cbaea1bba4726989b798c1b655731,Min/Max values,from xgboost import XGBRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
2,1,mlflow_demo2,0.377560,RandomForestRegressor,NaN,standardized values,NaN,NaN,NaN,7.0,NaN,279,13,aa9478ceea7e4cfbad36b01f14fcd769,standardized values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
3,1,mlflow_demo2,0.377378,RandomForestRegressor,NaN,Min/Max values,NaN,NaN,NaN,7.0,NaN,279,13,e15aa009827041de922aae35b88a6376,Min/Max values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
0,1,mlflow_demo2,0.377038,RandomForestRegressor,NaN,raw values,NaN,NaN,NaN,7.0,NaN,279,13,04435977636f4e86b7c9befcb6e562f5,raw values,from sklearn.ensemble import RandomForestRegre...,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
1,1,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,raw values,NaN,NaN,NaN,7.0,NaN,279,13,2b9949cae44e45b7961459deaa89eaa4,raw values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
11,1,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,standardized values,NaN,NaN,NaN,7.0,NaN,279,13,a0b7848f8d2247009ab234698c7c2796,standardized values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
13,1,mlflow_demo2,0.341047,ExtraTreesRegressor,NaN,Min/Max values,NaN,NaN,NaN,7.0,NaN,279,13,1b30e4b831eb420184f2506a7bbe8bc2,Min/Max values,from sklearn.ensemble import ExtraTreesRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL
9,1,mlflow_demo2,0.315244,MLPRegressor,NaN,standardized values,"(10, 10)",NaN,NaN,NaN,1343,NaN,13,b7863ba8b7c44c5b90d65432bd15a56b,standardized values,from sklearn.neural_network import MLPRegressor,/opt/conda/lib/python3.7/site-packages/ipykern...,LOCAL


In [14]:
df.shape

(16, 18)